# Train a hypergraph neural network using UniGCNII layers

This tutorial consists of three main steps:
1. Loading the CiCitationCora dataset and lifting it to the hypergraph domain.
2. Defining a hypergraph neural network (HGNN) which utlilizes the UniGCNII layer, and
3. Training the obtained neural network on the training data and evaluating it on test data.

First, we import the neccessary packages.


In [1]:
import torch
import pickle
import numpy as np
import scipy.sparse as sp

from topomodelx.nn.hypergraph.unigcnii import UniGCNII

torch.manual_seed(0)
np.random.seed(0)

If GPUs are available, we want to make use of them, otherwise the model is run on CPU.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Loading the data

We are using the Cora co-citation dataset. Here, the nodes represent documents and the edges in the graph represent documents which are co-cited. It is possible to compute this graph from the citation network directly. However, this is computationally very expensive. Instead, we load the dataset directly as it is available to download.

The task here is to classify each node and assign one of 7 possible classes to it. The dataset is a standard benchmark used in HGNN literature.

In [3]:
! wget https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/features.pickle
! wget https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/hypergraph.pickle
! wget https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/labels.pickle

--2023-11-03 14:49:01--  https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/features.pickle
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/malllabiisc/HyperGCN/master/data/cocitation/cora/features.pickle [following]
--2023-11-03 14:49:01--  https://raw.githubusercontent.com/malllabiisc/HyperGCN/master/data/cocitation/cora/features.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404937 (395K) [application/octet-stream]
Saving to: ‘features.pickle.1’

features.pickle.1   100%[===================>] 395.45K  --.-KB/s    in 0.04s   

2023-11-03 14:49:02 (10.7 MB/s) - ‘fea

In [4]:
! wget https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/splits/1.pickle

--2023-11-03 14:49:04--  https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/splits/1.pickle
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/malllabiisc/HyperGCN/master/data/cocitation/cora/splits/1.pickle [following]
--2023-11-03 14:49:04--  https://raw.githubusercontent.com/malllabiisc/HyperGCN/master/data/cocitation/cora/splits/1.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51582 (50K) [application/octet-stream]
Saving to: ‘1.pickle.1’

1.pickle.1          100%[===================>]  50.37K  --.-KB/s    in 0.002s  

2023-11-03 14:49:04 (27.8 MB/s) - ‘1.pickle.1’ 

Now, we can load the loaded dataset

In [5]:
with open("features.pickle", "rb") as handle:
    features = pickle.load(handle).todense()

with open("hypergraph.pickle", "rb") as handle:
    hypergraph = pickle.load(handle)

with open("labels.pickle", "rb") as handle:
    labels = pickle.load(handle)

/tmp/ipykernel_670387/2369537045.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  features = pickle.load(handle).todense()


In [6]:
# transform the input and output features to pytorch
x_0 = sp.csr_matrix(np.array(features), dtype=np.float32)
x_0 = torch.FloatTensor(np.array(x_0.todense()))
x_0 = x_0.to(device)

y = torch.LongTensor(np.array(labels))
y = y.to(device)

# construct the incidence matrix
h = np.zeros((x_0.shape[0], len(hypergraph)))

for num, nodes in enumerate(hypergraph.values()):
    h[list(nodes), num] = 1

Finally, we add self-loops to the dataset i.e. for every node $v$, we add a hyper-edge only containing that specific node $e = \{ v \}$. This is the standard format expected by the GCNII layers and transform the matrix into a sparse pytorch tensor.

In [7]:
# add self loops
h2 = np.eye(x_0.shape[0])
incidence = np.hstack((h, h2))

# transform to pytorch
incidence = torch.Tensor(incidence).to_sparse_csr()
incidence = incidence.to(device)

/tmp/ipykernel_670387/2226475299.py:6: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  incidence = torch.Tensor(incidence).to_sparse_csr()


Now, we can load the predefine split in test and train data before we start constructing the HGNN.

In [8]:
# load the train-test split given by the dataset
with open("1.pickle", "rb") as H:
    splits = pickle.load(H)
    train, test = splits["train"], splits["test"]

train_idx = torch.LongTensor(train).to(device)
test_idx = torch.LongTensor(test).to(device)

## Creating a neural network

Define the network that initializes the base model and sets up the readout operation.
Different downstream tasks might require different pooling procedures.


In [9]:
class Network(torch.nn.Module):
    """ Network class that initializes the base model and readout layer.

    Base model parameters:
    ----------
    Reqired:
    in_channels : int
        Dimension of the input features.
    hidden_channels : int
        Dimension of the hidden features.

    Optitional:
    **kwargs : dict
        Additional arguments for the base model.
    
    Readout layer parameters:
    ----------
    out_channels : int
        Dimension of the output features.
    task_level : str
        Level of the task. Either "graph" or "node".        
    """
    def __init__(
            self, 
            in_channels, 
            hidden_channels,
            out_channels, 
            task_level="graph",
            **kwargs):
        super().__init__()
        
        # Define the model
        self.base_model = UniGCNII(
            in_channels=in_channels,
            hidden_channels=hidden_channels,
            **kwargs
        )


        # Readout
        self.linear = torch.nn.Linear(hidden_channels, out_channels)
        self.out_pool = True if task_level == "graph" else False
        
    def forward(self, x_0, incidence_1):
        # Base model
        x_0, x_1 = self.base_model(x_0, incidence_1)

        # Pool over all nodes in the hypergraph 
        if self.out_pool is True:
            x = torch.max(x_0, dim=0)[0]
        else:
            x = x_0

        return self.linear(x)

Initialize the model

In [10]:
# Base model hyperparameters
in_channels = x_0.shape[1]
hidden_channels = 128
n_layers=2
mlp_num_layers=1
input_drop=0.5

# Readout hyperparameters
out_channels = torch.unique(y).shape[0]
task_level = "graph" if out_channels==1 else "node"


model = Network(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels,
    n_layers=n_layers,
    input_drop=input_drop,
    task_level=task_level,
    ).to(device)

## Training the neural network

First, we specify the hyperparameters of the training process.

In [11]:
num_epochs = 30
test_interval = 5

Next, we can generate the corresponding model, optimizer, and loss function with corresponding sizes.

In [12]:
# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Categorial cross-entropy loss
loss_fn = torch.nn.CrossEntropyLoss()

# Accuracy
acc_fn = lambda y, y_hat: (y == y_hat).float().mean()

Now, we are ready to train the created model and evaluate the performance on the validation set.

In [13]:

for epoch in range(num_epochs):
    # set model to training mode
    model.train()

    y_hat = model(x_0, incidence)
    loss = loss_fn(y_hat[train_idx], y[train_idx])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


    if epoch % test_interval == 0:
        
        model.eval()
        y_hat = model(x_0, incidence)

        train_loss = loss_fn(y_hat[train_idx], y[train_idx])
        loss = loss_fn(y_hat[test_idx], y[test_idx])
        print(f"Epoch: {epoch} \
              Train_loss: {train_loss:.4f}, Train_acc: {acc_fn(y_hat[train_idx].argmax(1), y[train_idx]):.4f} \
              Test_loss: {loss:.4f}, Test_acc: {acc_fn(y_hat[test_idx].argmax(1), y[test_idx]):.4f}",
              flush=True)

Epoch: 0               Train_loss: 1.8349, Train_acc: 0.5000               Test_loss: 1.8957, Test_acc: 0.2819
Epoch: 5               Train_loss: 0.3446, Train_acc: 1.0000               Test_loss: 1.0406, Test_acc: 0.7099
Epoch: 10               Train_loss: 0.0204, Train_acc: 1.0000               Test_loss: 1.2339, Test_acc: 0.7079
Epoch: 15               Train_loss: 0.0401, Train_acc: 0.9929               Test_loss: 3.0919, Test_acc: 0.6223
Epoch: 20               Train_loss: 0.0021, Train_acc: 1.0000               Test_loss: 3.9843, Test_acc: 0.6083
Epoch: 25               Train_loss: 0.0006, Train_acc: 1.0000               Test_loss: 3.0639, Test_acc: 0.6748
